In [1]:
import os
from collections import defaultdict
from tqdm import tqdm

fname="Books"
path = "/DATA/Recommendation/amazon"

In [2]:
import json
def open_json(file_path):
    with open(file_path, "r") as f:
        dict_list = [json.loads(line[:-1]) for line in f.readlines()]
    return dict_list
meta = open_json(os.path.join(path, f"meta_{fname}.jsonl"))
data = open_json(os.path.join(path, f"{fname}.jsonl"))

In [2]:
txt = open(os.path.join(path, f"train.txt"), "r").readlines()
train = dict()
for line in tqdm(txt):
    u, history = line.rstrip().split(":")
    history = [int(i) for i in history.split(" ")]
    u = int(u)
    train.update({u: history})

100%|██████████| 1150206/1150206 [00:05<00:00, 218523.57it/s]


In [3]:
len(train)

1150206

In [12]:
def get_path(fname: str, ftype: str, folder: str):
    file_path = os.path.join(folder, f"{fname}.{ftype}")
    meta_path = os.path.join(folder, f"meta_{fname}.{ftype}")
    return file_path, meta_path

def preprocess(fname, ftype="jsonl", folder="/DATA/Recommendation/amazon", save=False, data_list=None, meta_list=None):
    countU = defaultdict(lambda: 0)
    countP = defaultdict(lambda: 0)
    line = 0

    usermap = dict()
    usernum = 0
    itemmap = dict()
    itemnum = 0
    User = dict()
    # review_dict = {}
    # name_dict = {"title": {}, "description": {}}
    if fname == "Books":
        name_dict = {
            "title": {},
            "author": {},
            "average_rating": {},
            "features": {},
            "categories": {},
        }

    file_path, meta_path = get_path(fname, ftype, folder)

    if data_list is None:
        print(f"Loading jsonl files..")
        data_list = open_json(file_path)
    if meta_list is None:
        meta_list = open_json(meta_path)


    if ftype == "jsonl":
        # counting interactions for each user and item

        for l in tqdm(data_list):
            line += 1
            asin = l["asin"]
            pasin = l["parent_asin"]
            rev = l["user_id"]
            time = l["timestamp"]
            countU[rev] += 1
            countP[asin] += 1

        meta_dict = {}
        for l in meta_list:
            meta_dict[l["parent_asin"]] = l

        for l in tqdm(data_list):
            line += 1
            asin = l["asin"]
            pasin = l["parent_asin"]
            rev = l["user_id"]
            time = l["timestamp"]

            threshold = 5
            if ("Beauty" in fname) or ("Toys" in fname):
                threshold = 4

            if countU[rev] < threshold or countP[asin] < threshold:
                continue

            if rev in usermap:
                userid = usermap[rev]
            else:
                usernum += 1
                userid = usernum
                usermap[rev] = userid
                User[userid] = []

            if asin in itemmap:
                itemid = itemmap[asin]
            else:
                itemnum += 1
                itemid = itemnum
                itemmap[asin] = itemid
            
            User[userid].append([time, itemid])

            
            for key in name_dict.keys():

                if key in ["title", "average_rating"]:
                    name_dict[key][itemmap[asin]] = meta_dict[pasin].get(key, "None")

                elif key == "author":
                    try:
                        author_name = meta_dict[pasin][key].get("name", "Unknown")
                        author_info = meta_dict[pasin][key].get("about", "None")
                        if isinstance(author_info, list):
                            author_info = " ".join(author_info)
                        name_dict[key][
                            itemmap[asin]
                        ] = f"name: {author_name}, about: {author_info}"
                    except:
                        name_dict[key][itemmap[asin]] = "None"
                elif key == "features":
                    features = meta_dict[pasin].get(key, "None")
                    if isinstance(features, list):
                        name_dict[key][itemmap[asin]] = " ".join(features)
                    else:
                        name_dict[key][itemmap[asin]] = features

                elif key == "categories":
                    categories = meta_dict[pasin].get(key, "None")
                    if isinstance(categories, list):
                        name_dict[key][itemmap[asin]] = ", ".join(categories)
                    else:
                        name_dict[key][itemmap[asin]] = categories
            
    if save:
        json.dump(
            name_dict,
            open(os.path.join(folder,f"{fname}_meta_name_dict.json"), "w"),
        )
        print(f"Saved {fname}_meta_name_dict.json")
        for userid in User.keys():
            User[userid].sort(key=lambda x: x[0])

        print(usernum, itemnum)

        f = open(os.path.join(folder, f"{fname}.txt"), "w", encoding="UTF-8")
        for user in User.keys():
            for i in User[user]:
                f.write("%d %d\n" % (user, i[1]))
        f.close()
        print(f"Saved {fname}.txt")

        # usermap, itemmap save
        for map_, name in zip([usermap, itemmap], ["user", "item"]):
            with open(os.path.join(f"{fname}_{name}map.json"), "w") as f:
                json.dump(map_, f)
            print(f"Saved {fname}_{name}map.json")

    
    return User, countU, countP


User, countU, countP = preprocess(fname=fname, ftype='jsonl', folder=path, data_list = data, meta_list=meta, save=True)

100%|██████████| 29475453/29475453 [02:56<00:00, 166902.10it/s]


Saved Books_meta_name_dict.json
1206660 980848
Saved Books.txt


In [72]:
def preprocess(
    fname,
    ftype="jsonl",
    folder="/DATA/Recommendation/amazon",
    save=False,
    data_list=None,
    meta_list=None,
):
    countU = defaultdict(lambda: 0)
    countP = defaultdict(lambda: 0)
    line = 0

    usermap = dict()
    usernum = 0
    itemmap = dict()
    itemnum = 0
    User = dict()
    # review_dict = {}
    # name_dict = {"title": {}, "description": {}}
    if fname == "Books":
        name_dict = {
            "title": {},
            "author": {},
            "average_rating": {},
            "features": {},
            "categories": {},
        }

    file_path, meta_path = get_path(fname, ftype, folder)

    if data_list is None:
        print(f"Loading jsonl files..")
        data_list = open_json(file_path)
    if meta_list is None:
        meta_list = open_json(meta_path)

    if ftype == "jsonl":
        # counting interactions for each user and item

        for l in tqdm(data_list):
            line += 1
            asin = l["asin"]
            pasin = l["parent_asin"]
            rev = l["user_id"]
            time = l["timestamp"]
            countU[rev] += 1
            countP[asin] += 1

        meta_dict = {}
        for l in meta_list:
            meta_dict[l["parent_asin"]] = l

        for l in tqdm(data_list):
            line += 1
            asin = l["asin"]
            pasin = l["parent_asin"]
            rev = l["user_id"]
            time = l["timestamp"]

            threshold = 5
            if ("Beauty" in fname) or ("Toys" in fname):
                threshold = 4

            if countU[rev] < threshold or countP[asin] < threshold:
                continue

            if rev in usermap:
                userid = usermap[rev]
            else:
                usernum += 1
                userid = usernum
                usermap[rev] = userid
                User[userid] = []

            if asin in itemmap:
                itemid = itemmap[asin]
            else:
                itemnum += 1
                itemid = itemnum
                itemmap[asin] = itemid

            if [time, itemid] not in User[userid]:
                User[userid].append([time, itemid])

                for key in name_dict.keys():

                    if key in ["title", "average_rating"]:
                        name_dict[key][itemmap[asin]] = meta_dict[pasin].get(
                            key, "None"
                        )

                    elif key == "author":
                        try:
                            author_name = meta_dict[pasin][key].get("name", "Unknown")
                            author_info = meta_dict[pasin][key].get("about", "None")
                            if isinstance(author_info, list):
                                author_info = " ".join(author_info)
                            name_dict[key][
                                itemmap[asin]
                            ] = f"name: {author_name}, about: {author_info}"
                        except:
                            name_dict[key][itemmap[asin]] = "None"
                    elif key == "features":
                        features = meta_dict[pasin].get(key, "None")
                        if isinstance(features, list):
                            name_dict[key][itemmap[asin]] = " ".join(features)
                        else:
                            name_dict[key][itemmap[asin]] = features

                    elif key == "categories":
                        categories = meta_dict[pasin].get(key, "None")
                        if isinstance(categories, list):
                            name_dict[key][itemmap[asin]] = ", ".join(categories)
                        else:
                            name_dict[key][itemmap[asin]] = categories
            else:
                continue

    if save:
        # json.dump(
        #     name_dict,
        #     open(os.path.join(folder, f"{fname}_meta_name_dict.json"), "w"),
        # )
        # print(f"Saved {fname}_meta_name_dict.json")
        for userid in User.keys():
            User[userid].sort(key=lambda x: x[0])

        print(usernum, itemnum)

        f = open(os.path.join(folder, f"{fname}.txt"), "w", encoding="UTF-8")
        for user in User.keys():
            for i in User[user]:
                f.write("%d %d\n" % (user, i[1]))
        f.close()
        print(f"Saved {fname}.txt")

        # # usermap, itemmap save
        # for map_, name in zip([usermap, itemmap], ["user", "item"]):
        #     with open(os.path.join(f"{fname}_{name}map.json"), "w") as f:
        #         json.dump(map_, f)
        #     print(f"Saved {fname}_{name}map.json")

preprocess(fname=fname, ftype='jsonl', folder=path, data_list = data, meta_list=meta, save=True)

100%|██████████| 29475453/29475453 [04:21<00:00, 112625.95it/s]


1206660 980848
Saved Books.txt


2. data partition

In [10]:
import sys
sys.path.append("/home/doyooni303/experiments/LLMRec/ReLLMRec")
from src.dataset.utils import data_partition
user_train, user_valid, user_test = data_partition(fname=fname, path=path)

splitting data by user: 100%|██████████| 1206660/1206660 [00:01<00:00, 628042.36it/s]


In [3]:
!python3 /home/doyooni303/experiments/LLMRec/ReLLMRec/src/dataset/get_similar_users.py

Books.txt exists
Preprocessing history: 100%|█████| 1206660/1206660 [00:00<00:00, 1341253.01it/s]
Initializing UserSimilarityFinder...
Finding similar users for 548840 query users...
Processing batches: 100%|█████████████████████| 536/536 [13:19<00:00,  1.49s/it]
Saving results...
All completed !


similar user 구한 후

In [5]:
import json
similar_users = json.load(open(os.path.join(path,f"{fname}_similar_users.json"), "r"))
itemmap = json.load(open(os.path.join(path, f"{fname}_itemmap.json"), "r"))
usermap = json.load(open(os.path.join(path, f"{fname}_usermap.json"), "r"))
idx2user = {v: k for k, v in usermap.items()}
idx2item = {v: k for k, v in itemmap.items()}

# train, valid, test = data_partition(fname, "/home/doyooni303/experiments/LLMRec/data/amazon/Books")

In [7]:
txt = open(os.path.join(path, f"{fname}.txt"), "r").readlines()

USER = defaultdict(list)
for line in tqdm(txt):
    u, i = line.rstrip().split(" ")
    u = int(u)
    i = int(i)
    USER[u].append(i)

# for SELFRec data save
max_length = 20
train,valid, test = [], [], []
for user, h_list in tqdm(USER.items(), desc="splitting data by user"):
    if len(h_list) >= 3:
        train.append(f"{user}:{' '.join([str(i) for i in h_list[-(max_length+2):-2]])}"+"\n")
        valid.append(f"{user}:{h_list[-2]}"+"\n")
        test.append(f"{user}:{h_list[-1]}\n")
    else:
        pass

splitting data by user: 100%|██████████| 1206660/1206660 [00:04<00:00, 256314.97it/s]


In [8]:
from typing import List
import os
def write_text_file(file_path: str, data: List):
  with open(file_path, 'w') as f:
    f.writelines(data)

write_text_file(os.path.join(path, "train.txt"), train)
write_text_file(os.path.join(path, "test.txt"), test)

In [12]:
meta_name_dict = json.load(open(os.path.join(path, f"{fname}_meta_name_dict.json"), "r"))
query_user_idx, _ = list(user_train.items())[10]

query_similar_users = similar_users.get(str(query_user_idx), "None")
if isinstance(query_similar_users, list):
    query_similar_users = [idx for idx, _ in query_similar_users]

similar_users_history = {}
for idx in query_similar_users:
    sequence = user_train.get(idx, [])
    
    sequence_titles = [meta_name_dict['title'][str(i)] for i in sequence]
    similar_users_history.update({idx: sequence_titles})
similar_users_history

{693844: ["Bloodborne Collector's Edition Strategy Guide",
  'The Poky Little Puppy (A Little Golden Book Classic)',
  'The Monster at the End of This Book',
  'Night of the Hunter: Companions Codex, I',
  'The Companions: The Sundering, Book I'],
 1058837: ['Baby Animals (Little Golden Book)',
  'Dumbo (Disney Classic) (Little Golden Book)',
  'Tawny Scrawny Lion (Little Golden Book)',
  'The Little Red Hen (Little Golden Book)',
  'The Poky Little Puppy (A Little Golden Book Classic)'],
 135572: ['Galaxy S5 Case, GreatShield [TACT | Quill Design] SLIM FIT Pattern Rubber Coating Embossed Snap On Case Back Cover for Samsung Galaxy S5 SV (Gold)',
  'The Three Bears',
  'The Three Little Pigs (Disney Classic) (Little Golden Book)',
  'The Poky Little Puppy (A Little Golden Book Classic)',
  'The Monster at the End of This Book',
  'The Phantom of the Opera'],
 151622: ['Head, Shoulders, Knees and Toes...',
  'The Pout-Pout Fish',
  "Giraffes Can't Dance (Board Book)",
  'The Wonderful Th